In [31]:
import pandas as pd
import os
from pathlib import Path
import re

In [27]:
#inputs
inputdir = input("Directory: ")
organism = input("Organism (capitalize first lettter): ")
cell_line = input("Cell line: ")
oxidant = input("Oxidant (if none: press enter): ")
ox_dose = input("Oxidant dose in SPECIFY UNIT HERE (if none, enter 0): ")
treatment = input("Secondary treatment (if none: press enter): ")
treatment_dose = input("Secondary treatment dose in SPECIFY UNIT HERE (if none, enter 0): ")
time_ox = input("Length of oxidant treatment in minutes (if none, enter 0): ")
time_tr = input("Length of secondary treatment in minutes (if none, enter 0): ")
extra = input("Extra notes? (if none: press enter)")

In [28]:
multiline_str = """The metadata for the result files of {} are the following:
The organism is {}.
The cell line is {}.
The oxidant treatment is {}.
The {} dose is {} SPECIFY UNIT HERE.
The secondary treatment is {}.
The {} dose is {} SPECIFY UNIT HERE.
The length of oxidant treatment is {} minutes.
The length of secondary treatment is {} minutes.
Extra notes: {}.""".format(inputdir, organism, cell_line, oxidant, oxidant, ox_dose, treatment, treatment, treatment_dose, time_ox, time_tr, extra)
print(multiline_str)

The metadata for the result files of /home/robbe/ionbot/PXD018549/ionbot_files/qe1_2019oct05_05_uc190822_dw35_1c are the following:
The organism is Human.
The cell line is HeLa.
The oxidant treatment is H2O2.
The H2O2 dose is 5 SPECIFY UNIT HERE.
The secondary treatment is .
The  dose is  SPECIFY UNIT HERE.
The length of oxidant treatment is 0 minutes.
The length of secondary treatment is 0 minutes.
Extra notes: .


In [ ]:
dir = "Processed"
path = os.path.join(inputdir, dir)
os.mkdir(path)


In [36]:
txt = inputdir
x = re.findall(".*\/",txt)
file_name = inputdir.removeprefix(x[0])
print("The file name is: {}".format(file_name))
y = re.compile('') #ROBBE HIER VERDER WERKEN? PROBEER UNIPROT ACCESSIE NUMMER ER UIT TE HALEN

['/home/robbe/ionbot/PXD018549/ionbot_files/']
qe1_2019oct05_05_uc190822_dw35_1c


In [48]:
from cmath import nan

ionbot_first = pd.read_csv(Path("{}/ionbot.first.csv".format(inputdir)))
ionbot_first_proteins = pd.read_csv(Path("{}/ionbot.first.proteins.csv".format(inputdir)))
ionbot_lower = pd.read_csv(Path("{}/ionbot.lower.csv".format(inputdir)))
ionbot_modifications = pd.read_csv(Path("{}/ionbot.modifications.csv".format(inputdir)))
ionbot_coeluting_proteins = pd.read_csv(Path("{}/ionbot.coeluting.proteins.csv".format(inputdir)))
files = [ionbot_first, ionbot_first_proteins, ionbot_lower, ionbot_modifications, ionbot_coeluting_proteins]
for i in files:
    copy_df = i.copy()
    copy_df['Organism'] = organism
    copy_df['Cell line'] = cell_line
    if oxidant != "":
        copy_df['Oxidant treatment'] = oxidant
    else:
        copy_df['Oxidant treatment'] = nan
    copy_df['Oxidant dose (SPECIFY DOSE HERE)'] = ox_dose
    if treatment != "":
        copy_df['Secondary treatment'] = treatment
    else: 
        copy_df['Secondary treatment'] = nan
    copy_df['Secondary treatment dose (SPECIFY UNIT HERE)'] = treatment_dose
    copy_df['Length of oxidant treatment (mins)'] = time_ox
    copy_df['Length of Secondary treatment (mins)'] = time_tr
    if extra != "":
        copy_df['Notes'] = extra 
    else: 
        copy_df['Notes'] = nan
    copy_df['spectrum_file'] = file_name
    if i.all in [ionbot_first.all, ionbot_first_proteins.all]:
        copy_df['First/Lower/Coeluting'] = "First"
    elif i.all == ionbot_lower.all:
        copy_df['First/Lower/Coeluting'] = "Lower"
    elif i.all == ionbot_coeluting_proteins.all:
        copy_df['First/Lower/Coeluting'] = "Coeluting"
    
    
    

  ionbot_match_id                                  spectrum_title   scan  \
0        0_8228_1  controllerType=0 controllerNumber=1 scan=10000  10000   
1        0_8231_1  controllerType=0 controllerNumber=1 scan=10003  10003   
2        0_8232_3  controllerType=0 controllerNumber=1 scan=10004  10004   
3        0_8235_3  controllerType=0 controllerNumber=1 scan=10007  10007   
4        0_8236_1  controllerType=0 controllerNumber=1 scan=10008  10008   

                       spectrum_file  precursor_mass  peptide_mass  \
0  qe1_2019oct05_05_uc190822_dw35_1c      1044.54526    10465.4298   
1  qe1_2019oct05_05_uc190822_dw35_1c       984.49975     9845.0284   
2  qe1_2019oct05_05_uc190822_dw35_1c       992.52671     9925.2907   
3  qe1_2019oct05_05_uc190822_dw35_1c      1847.92462    18479.2646   
4  qe1_2019oct05_05_uc190822_dw35_1c      1464.66562    14646.6670   

   observed_retention_time  charge   database_peptide    matched_peptide  ...  \
0               1435.58130       2       